In [1]:
import localutils as lu
import numpy as np
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.utils.validation import check_array

Implement the evidence_matrix and evidence_array methods in localutils. See the lecture for details.

# Toy test 1

In [2]:
X = [[1, 2, 3], [-1, 2, 3], [1, -2, 3], [-1, 2, 3]]
X = check_array(X, accept_sparse="csr")

In [3]:
X

array([[ 1,  2,  3],
       [-1,  2,  3],
       [ 1, -2,  3],
       [-1,  2,  3]])

In [6]:
X.shape

(4, 3)

In [7]:
w = np.asarray([-2, 3, -4])

In [8]:
w

array([-2,  3, -4])

In [9]:
w.shape

(3,)

In [6]:
M = lu.evidence_matrix(X, w)

In [7]:
M

array([[ -2,   6, -12],
       [  2,   6, -12],
       [ -2,  -6, -12],
       [  2,   6, -12]])

In [8]:
ea = lu.evidence_array(M)

In [9]:
ea

[[-14, 6], [-12, 8], [-20, 0], [-12, 8]]

# Toy test 2

Create your own X and w to test your code.

In [10]:
X = [[1, 2, 3], [-1, 2, 3], [-1, -2, 3], [-1, -2, -3]]
X = check_array(X, accept_sparse="csr")
X

array([[ 1,  2,  3],
       [-1,  2,  3],
       [-1, -2,  3],
       [-1, -2, -3]])

In [11]:
w = np.asarray([-4, -5, -6])
w

array([-4, -5, -6])

In [12]:
M = lu.evidence_matrix(X, w)
M

array([[ -4, -10, -18],
       [  4, -10, -18],
       [  4,  10, -18],
       [  4,  10,  18]])

In [13]:
ea = lu.evidence_array(M)
ea

[[-32, 0], [-28, 4], [-18, 14], [0, 32]]

# Real data

    1. Load boston house prices dataset

In [14]:
from sklearn.datasets import load_boston
boston = load_boston()
X = boston.data
y = boston.target
feature_names = boston.feature_names

    2. Z-score it using StandardScaler from scikit-learn

In [15]:
from sklearn.preprocessing import StandardScaler
scalar = StandardScaler()
scalar.fit(X,y)
Xz = scalar.transform(X)

    3. Fit a Lasso regression model

In [16]:
from sklearn.linear_model import Lasso
m = Lasso()
m.fit(Xz, y)

Lasso(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=1000,
   normalize=False, positive=False, precompute=False, random_state=None,
   selection='cyclic', tol=0.0001, warm_start=False)

    4. Compute the evidence matrix; use the coefficients of Lasso as the weights

In [17]:
w = m.coef_
em = lu.evidence_matrix(Xz,w)

    5. Compute the evidence array

In [18]:
ea = lu.evidence_array(em)

    6. Print "index, negative evidence, positive evidence" for top 10 items where negative evidence + positive evidence is close to zero.

In [19]:
print("INDEX","\t ","NEGATIVE EVIDENCE","\t","POSITIVE EVIDENCE")
print("----------------------------------------------------")
for s in sorted(range(len(ea)), key = lambda x: abs(ea[x][1] + ea[x][0]))[:10]:
    print(s,"\t",ea[s][0],"\t",ea[s][1])

INDEX 	  NEGATIVE EVIDENCE 	 POSITIVE EVIDENCE
----------------------------------------------------
355 	 -3.549528710486456 	 3.556504027097464
230 	 -1.173730655729564 	 1.1969217303157698
79 	 -1.7392145444564937 	 1.8429125478422939
155 	 -2.2213931180889217 	 2.332962150130343
505 	 -2.564955253489921 	 2.450539908788513
485 	 -1.0836739935378703 	 1.1988357814574857
37 	 -2.1425923541816614 	 2.0084865826443306
78 	 -0.3553278957273713 	 0.2144736150458669
16 	 -2.932187744074467 	 3.076295839794152
345 	 -1.2601504786474012 	 1.1119730453916121


    7. Among top 10, which one has the minimum of the maximum of the absolute value of the evidences?

In [20]:
for t in sorted(range(len(ea)), key = lambda x: abs(ea[x][1] + ea[x][0]))[:10]:
    if max(abs(ea[t][0]),abs(ea[t][1])) < max(abs(ea[s][0]),abs(ea[s][1])):
        s = t
print(s,"\t",ea[s][0],"\t",ea[s][1])

78 	 -0.3553278957273713 	 0.2144736150458669


    8. Among top 10, which one has the maximum of the minimum of the absolute value of the evidences?

In [21]:
for t in sorted(range(len(ea)), key = lambda x: abs(ea[x][1] + ea[x][0]))[:10]:
    if min(abs(ea[t][0]),abs(ea[t][1])) > min(abs(ea[s][0]),abs(ea[s][1])):
        s = t
print(s,"\t",ea[s][0],"\t",ea[s][1])

355 	 -3.549528710486456 	 3.556504027097464


    9. Discuss your findings.

It is evident that the maximum of the minimums and minimum of the maximums to be at different indicies. Furthermore, it is interesting that the maximum of the minimums were the largest values independent of the minimum comparison; this is due to finding the top 10 forces the results of neg and pos evidence to be close to each other 